In [1]:
import geopandas as geo
import pandas as pd
from shapely.geometry import Point
import os

### Read in census tracts shapefile

In [2]:
tracts = geo.read_file('LA_CensusTracts/CENSUS_TRACTS_2000.shp')

tracts.head()

,CT00,LABEL,Shape_STAr,Shape_STLe,geometry
0,101110,1011.10,1.228537e+07,15056.954289,"POLYGON ((6475581.002129614 1916939.626698852,..."
1,101120,1011.20,1.364394e+08,70339.952869,"POLYGON ((6471659.999801025 1919631.501564443,..."
2,101210,1012.10,7.019669e+06,12082.404482,"POLYGON ((6473070.498292357 1913983.375393525,..."
3,101220,1012.20,7.485860e+06,12652.916695,"POLYGON ((6477593.248105109 1914091.177999437,..."
4,101300,1013.00,2.784498e+07,28343.215220,"POLYGON ((6480645.62028119 1906746.72648102, 6..."


### Read family data

- Be sure to read 'familyid' as a string or initial zeroes will be lost!
- Only reading in some of the columns for this operation

In [3]:
data_cols = ['familyid','longitude','latitude']
eth = pd.read_csv('la_family_data.csv', sep=',', usecols=data_cols, dtype={'familyid': 'str'})

eth.head()

,familyid,latitude,longitude
0,000000064097,34.13613,-118.14349
1,000000191203,33.84097,-118.06158
2,000000281697,34.69933,-118.13906
3,000000374686,33.79798,-118.11764
4,000000440577,34.40821,-118.44098


### Lat/Lon pairs need to be converted to a Point() data type

We'll put these in a new column called `coordinates`

In [4]:
eth['coordinates'] = list(zip(eth['longitude'], eth['latitude']))
eth['coordinates'] = eth['coordinates'].apply(Point)

eth.head()

,familyid,latitude,longitude,coordinates
0,000000064097,34.13613,-118.14349,POINT (-118.14349 34.13613)
1,000000191203,33.84097,-118.06158,POINT (-118.06158 33.84097)
2,000000281697,34.69933,-118.13906,POINT (-118.13906 34.69933)
3,000000374686,33.79798,-118.11764,POINT (-118.11764 33.79798)
4,000000440577,34.40821,-118.44098,POINT (-118.44098 34.40821)


### Convert to GeoDataFrame, specifying which column contains the geometry

- Have to set the Coordinate Reference System (CRS) initialy to say it's been hand-coded lat/lon, `epsg:4326`
- Then reproject to same CRS as tracts

In [5]:
gdf = geo.GeoDataFrame(eth, geometry='coordinates')
gdf.crs = {'init': 'epsg:4326'}
gdf = gdf.to_crs(tracts.crs)

### Do the spatial join

- We'll do a "left" join so we don't lose any points if some of our families don't happen to fall within a census tract – we'll just get a NULL (None) in the CT100 field.
- 'within' just slightly faster operation than 'intersects' or 'contains' – For points in a polygon it shouldn't matter which operation you use

In [6]:
eth_tracts = geo.sjoin(gdf, tracts, how="left", op='within')

eth_tracts[['familyid','CT00']].head()

,familyid,CT00
0,000000064097,463600
1,000000191203,555103
2,000000281697,900806
3,000000374686,574300
4,000000440577,920043


### Saving only the two fields that we need to output CSV

In [7]:
eth_tracts[['familyid','CT00']].to_csv('la_family_tracts.csv', index=False, encoding='utf-8')

### Alternatively save data to shapefile for easier mapping

I like to put shapefiles in their own directory since a "shapefile" really consists of multiple files, and this makes it easier to keep them together.

- *Creating the directory will give an error if it already exists, so I check whether it's already been created first.*
- *This also happens to be an easy way to create a DBF file of your data!*

In [8]:
if not os.path.exists('la_family_SHP'):
	os.mkdir('la_family_SHP')
    
eth_tracts[['familyid', 'coordinates', 'CT00']].to_file('la_family_SHP/la_family_points.shp')